In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
import numpy as np
import time
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# number of bsm to be analized
window = '02'

simPath = "/content/drive/MyDrive/dataset/veremi/veremiWithT2/simulationscsv"
bsmPath = "/content/drive/MyDrive/globecom/"+window+"bsm"
preprocPath = bsmPath+"/preprocessing"
timePath = preprocPath+"/time"

paths = [simPath, bsmPath, preprocPath, timePath]
for p in paths:
  if os.path.exists(p) == False:
    os.mkdir(p)
  print(os.path.exists(p), p)
# os.chdir(articlePath)
# os.listdir()

True /content/drive/MyDrive/dataset/veremi/veremiWithT2/simulationscsv
True /content/drive/MyDrive/globecom/02bsm
True /content/drive/MyDrive/globecom/02bsm/preprocessing
True /content/drive/MyDrive/globecom/02bsm/preprocessing/time


In [3]:
np.seterr('raise')
# separar comunicações completas por veículos
os.chdir(simPath)
simulations = pd.Series([f for f in os.listdir() if os.path.isfile(f)]).sort_values().reset_index(drop=True)[:]
tlist = []
for simul in simulations.values:
  t = simul.split('.')
  tlist.append(int(t[0][-3:]))
totalsim = len(simulations)
os.chdir(preprocPath)
if os.listdir():
  clist = []
  concluded = pd.Series([f for f in os.listdir() if os.path.isfile(f)], dtype=pd.StringDtype()).sort_values().reset_index(drop=True)
  for concl in concluded.values:
    c = concl.split('.')
    clist.append(int(c[0][-3:]))
else:
  clist = []
remainlist = list(set(tlist) - set(clist))
while remainlist:
  os.chdir(preprocPath)
  concluded = pd.Series([f for f in os.listdir() if os.path.isfile(f)], dtype=pd.StringDtype()).sort_values().reset_index(drop=True)
  clist = []
  for concl in concluded.values:
    c = concl.split('.')
    clist.append(int(c[0][-3:]))
  remainlist = list(set(tlist) - set(clist))
  if remainlist:
    szremainlist = len(remainlist)
    item = 0
    sim = remainlist[item]
    # start preprocess
    start_time = time.time()
    os.chdir(simPath)
    data = pd.read_csv(simulations[sim], index_col=0)
    # # filtrar apenas as colunas que precisamos
    # data = data[['receiver', 'sender']]
    a = pd.Series(data['receiver'].unique().astype(int)).sort_values().reset_index(drop=True)
    b = pd.Series(data['sender'].unique().astype(int)).sort_values().reset_index(drop=True)
    vehicles = pd.concat([a, b], ignore_index=True).unique()
    size = len(vehicles)
    newdata = []
    dataset = []

    for idr, r in enumerate(vehicles):
      # if idr <= 0:
        for ids, s in enumerate(data['sender'].loc[data['receiver'] == r].astype(int).unique()):
          # if ids <= 0:
            print('\r',
                  'Processing simulation '+str(remainlist[item]),
                  ', vehicle '+str(idr+1)+' of '+str(size),
                  " in {:.2f}".format(time.time() - start_time)+' sec |',
                  ' faltam '+str(szremainlist)+' simulações.',
                  sep='', end='', flush=True)
            interaction = data.loc[(data['receiver'] == r) & (data['sender'] == s)]
            atktype = interaction['attackerType'].unique()[0]
            # fix atk_16 records
            if atktype == 16:
              atk_16 = []
              for idind, ind in enumerate(interaction.index):
                pxSnd = interaction['pxSnd'][ind]
                pySnd = interaction['pySnd'][ind]
                sxSnd = interaction['sxSnd'][ind]
                sySnd = interaction['sySnd'][ind]
                if idind > 0:
                  if sxSndOld != 0 and sxSndOld != 0:
                    if pxSndOld == pxSnd and pySndOld == pySnd:
                      atk_16.append(ind)
                pxSndOld = pxSnd
                pySndOld = pySnd
                sxSndOld = sxSnd
                sySndOld = sxSnd
              # print(atk_16)
              interaction = interaction.loc[atk_16]
            l = len(interaction)
            if l >= int(window):
              # print(r, s, l, atktype)
              newdata = np.append(newdata, [sim, r, s])                
              for idind, ind in enumerate(interaction[0:int(window)].index):
                
                # feature 1
                # RSSI
                feat1 = interaction['RSSI'][ind]

                # feature 2
                # distance between sender and receiver
                pxRcv = interaction['pxRcv'][ind]
                pyRcv = interaction['pyRcv'][ind]
                pzRcv = interaction['pzRcv'][ind]
                pxSnd = interaction['pxSnd'][ind]
                pySnd = interaction['pySnd'][ind]
                pzSnd = interaction['pzSnd'][ind]
                pRcv = np.array((pxRcv, pyRcv, pzRcv))
                pSnd = np.array((pxSnd, pySnd, pzSnd))
                feat2 = np.linalg.norm(pRcv-pSnd)

                # feature3 conformidade de deslocamento
                sxSnd = interaction['sxSnd'][ind]
                sySnd = interaction['sySnd'][ind]
                szSnd = interaction['szSnd'][ind]
                sSnd = np.array((sxSnd, sySnd, szSnd))
                sendTime = interaction['sendTime'][ind]
                if idind == 0:
                  feat3 = np.nan
                if idind >= 1:
                  pSndNow = pSnd
                  pEstimated = pSndOld + (sSndOld * (sendTime - sendTimeOld))
                  feat3 = np.linalg.norm(pSndNow-pEstimated)
                pSndOld = pSnd
                sSndOld = sSnd
                sendTimeOld = sendTime
                                                  
                newdata = np.append(newdata, [feat1, feat2, feat3])

              newdata = np.append(newdata, atktype)
              dataset.append(newdata)
              newdata = []

                                

                  
    columns  = ['sim', 'receiver', 'sender']
    for i in range(int(window)):
      columns.append('RSSI'+str(i))
      columns.append('distance'+str(i))
      columns.append('conformity'+str(i))
    columns.append('attackerType')
    df = pd.DataFrame(dataset, columns=columns)
    # display(df)

    os.chdir(preprocPath)
    print('\rSaving sim'+f"{remainlist[item]:03}"+'.csv', sep='', end='', flush=True)
    if os.path.exists(preprocPath+'/sim'+f"{remainlist[item]:03}"+'.csv') == False:
      df.to_csv('sim'+f"{remainlist[item]:03}"+'.csv')
    
    # log the time of preprocessing
    os.chdir(timePath)
    simTime = time.time() - start_time
    data = {"simulation": "sim"+f"{remainlist[item]:03}", "time": simTime}
    try:
      timeDf = pd.read_csv('timePreprocessing.csv', index_col=0)
    except FileNotFoundError:
      timeDf = pd.DataFrame({"simulation": [data['simulation']], "time": [data['time']]})
      timeDf.to_csv('timePreprocessing.csv')
    else:
      if (timeDf['simulation'] == data['simulation']).any():
        timeDf.loc[timeDf['simulation'] == data['simulation'], ['time']] = data['time']
        timeDf.to_csv('timePreprocessing.csv')
      else:    
        timeDf = timeDf.append(data, ignore_index=True)
        timeDf.to_csv('timePreprocessing.csv')

    # retira o item da remainlist
    remainlist.pop(item)
    del clist

os.chdir(timePath)
timeDf = pd.read_csv('timePreprocessing.csv', index_col=0)
print('\rDone in ', "{:.2f}".format(timeDf['time'].sum())+' sec', sep='', end='', flush=True)

Done in 29969.92 sec